In [ ]:
import psycopg2
import re
from lxml import etree

infile = "testdump-s.xml"
lang = "en"

ns = '{http://www.mediawiki.org/xml/export-0.10/}'

# drop everything and start over?
drop = False

# Connect to an existing database
conn = psycopg2.connect("dbname=CPT user=carsten")

# Open a cursor to perform database operations
cur = conn.cursor()

# let's pre-compile some regexes
linkpattern = re.compile("\[\[?([^]|]*)(\|)?([^]|]*)?\]\]")

if drop: # drop everything and recreate tables
    cur.execute('DROP TABLE IF EXISTS "links";')
    cur.execute('CREATE TABLE "links" ("from" varchar, "to" varchar, "lang" varchar, "links" integer, "mentions" integer) ;')


    
def findreferences(pagetext):
    # find all links via regex, save in a dict with the link as key and number of occurrences for this link as value
    links = linkpattern.finditer(pagetext)
    for match in links:
        print match.group(0)
        
        title = match.group(1)
        print title
        
        if match.group(2):
            alias = match.group(3)
            print alias
    # 2. go through array of links; for every link: 
    #      a. find all mentions via regex (also for aliases!)
    #      b. insert into PostGIS links table
    
    
    
# for the parsing, we follow the approach explained here: 
# http://www.ibm.com/developerworks/xml/library/x-hiperfparse/ 
pages = etree.iterparse(infile, events=('end',), tag=ns+'page')

# go through wikipedia pages in dump, one by one:
for event, page in pages:    
    title    = page.find(ns+'title')
    pageid   = page.find(ns+'id')    
    pagetext = page.find(ns+'revision/'+ns+'text')
    
    print " >> "+title.text
    findreferences(pagetext.text)
    
    # print(title.text)
    # It's safe to call clear() here because no descendants will be accessed
    page.clear()

    # Also eliminate now-empty references from the root node to <Title> 
    while page.getprevious() is not None:
        del page.getparent()[0]



    
    
    
# Pass data to fill
# cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)",

# Query the database and obtain data as Python objects
#cur.execute("SELECT * FROM test;")
#cur.fetchone()


# Make the changes to the database persistent
conn.commit()

# Close communication with the database
cur.close()
conn.close()